## Output File Generation

This section processes the results from HADDOCK to extract essential information from PDB files and compute a HADDOCK score for each. It also selects the best model based on these scores.


In [ ]:
# Import necessary libraries
import numpy as np
import os
import pandas as pd
!pip install pdb2sql # Ensure pdb2sql is installed 
from pdb2sql import StructureSimilarity


# Define a function to convert string identifiers into a list based on underscores
def Convert(string):
    li = list(string.split("_"))
    return li


# Function to extract energy values and compute a HADDOCK score from a PDB file
def initial_matrix (id_uniprot, direction_pdb, name_subfichero ):
    # Open and read the PDB file
    f = open(direction_pdb, "r")
    listas_pdb = f.readlines()

    # Extract energy values from specific lines within the PDB file
    row_en = list(listas_pdb[6].split(", "))
    row_desol = list(listas_pdb[26].split(": "))
    row_bur = list(listas_pdb[31].split(": "))

    # Convert string values to float and calculate HADDOCK score
    Evdw = float(row_en[5])
    Eelec = float(row_en[6])
    Eair = float(row_en[7])
    Edesol = float(row_desol[1][:-1])
    Eburied = float(row_bur[1][:-1])

    # HADDOCK score-water =  1.0 Evdw + 0.2 Eelec + 1.0 Edesol +  0.1 Eair
    HHscore = round(Evdw + 0.2*Eelec + Edesol + 0.1*Eair, 3)

    #irmsd_value = irmsd(direction_pdb)
    
    # Create a core list from the name of the subfile (removing file extension)
    core = Convert(name_subfichero[:-4])

    core.append(id_uniprot)
    core.append(Evdw)
    core.append(Eelec)
    core.append(Eair)
    core.append(Edesol)
    core.append(Eburied)
    core.append(HHscore)
    #core.append(irmsd_value)
    f.close()

    return core

# Define a function to select the best model based on the mean HADDOCK scores across clusters
def select_best_temp_row(temp_matrix):

    # Convert list to DataFrame for easier manipulation
    matrix = pd.DataFrame(temp_matrix)
    # ['cluster10', '1', '001_A0A1S4NYE3', -37.2585, -195.878, 34.8024, -14.1129, 1508.18, -87.067]
    #  ['cluster10', '1', '001_A0A1S4NYE3', -37.2585, -195.878, 34.8024, -14.1129, 1508.18, -87.067]
    # ['cluster10', '2', '001_A0A1S4NYE3', -41.3461, -136.622, 3.5017, -12.3569, 1567.62, -80.677]
    # ['cluster10', '3', '001_A0A1S4NYE3', -28.0769, -187.567, 1.43022, -13.5289, 1297.53, -78.976]
    # ['cluster10', '4', '001_A0A1S4NYE3', -23.0408, -184.41, 6.20537, -6.61631, 972.419, -65.919]
    # ['cluster13', '1', '001_A0A1S4NYE3', -51.3389, -64.3856, 3.40222, -32.0151, 1772.5, -95.891]
    # ['cluster13', '2', '001_A0A1S4NYE3', -51.8539, -89.1169, 72.5337, -32.2768, 1961.31, -94.701]
    # ['cluster13', '3', '001_A0A1S4NYE3', -41.419, -78.1842, 37.2205, -28.981, 1495.74, -82.315]
    # ['cluster13', '4', '001_A0A1S4NYE3', -39.1275, -87.1129, 34.6435, -21.3687, 1651.89, -74.454]

    # Convert cluster identifiers to categorical data and convert scores to float for comparison
    matrix[0] = matrix[0].astype("category")
    matrix[8] = matrix[8].astype("float")

    # Extract unique category labels
    categ = list(matrix[0].cat.categories)

    # Initialize reference and best vectors to store best scores and corresponding data
    best_vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]  # [0]*17
    ref = [100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100]

    vector = []
    for i in categ:
        row_matrix = matrix.loc[matrix[0] == i]
        #print(row_matrix)

        list_mean = [round(item, 3) for item in list(row_matrix.mean())]
        list_std = [round(item, 3) for item in list(row_matrix.std())]
        vector = [i] + list_mean + list_std


        # [cluster01 308.5, -45.469, -111.789, 160.614, -12.582, 1802.868, -64.347, irmsd]
        if vector[8] < ref[8]:
            best_vector = vector
        else:
            pass
        ref = best_vector


    return best_vector

## Output File Structure

**Column Names and Descriptions:**

- `Name`: Identifier combining cluster information and UniProt ID.
- `ncluster`: Cluster number.
- `iduniprot`: UniProt ID of the protein.
- `Evdw`: Van der Waals energy.
- `sd_Evdw`: Standard deviation of the Van der Waals energy.
- `Eelec`: Electrostatic energy.
- `sd_Eelec`: Standard deviation of the electrostatic energy.
- `Eair`: Air energy term (specific to context).
- `sd_Eair`: Standard deviation of the air energy term.
- `Edesol`: Desolvation energy.
- `sd_Edesol`: Standard deviation of the desolvation energy.
- `Eburied`: Energy of buried surface area.
- `sd_Eburied`: Standard deviation of the buried surface area energy.
- `HHscore`: HADDOCK score.
- `sd_HHscore`: Standard deviation of the HADDOCK score.
- `irmsd`: Root-mean-square deviation of interface positions.

**Example Output Row:**

This example illustrates how each entry in the dataset might appear, based on the computed energies and other metrics:

```
['C123_P0AEU0', 'cluster12', 308.5, -46.203, -15.686, 13.695, -26.114, 1358.722, -74.084, 0.446, 7.782, 5.912, 14.698, 2.127, 78.504, 7.11, 0.094]
```

In [ ]:
# Define an empty list to store the best rows
best_matrix = [] 

#csvfile = open("")

# Reference PDB files for comparison
ref = ['seca_001.pdb',"seca_002.pdb","seca_003.pdb","seca_004.pdb",
           "seca_005.pdb","seca_006.pdb","seca_007.pdb","seca_008.pdb",
           "seca_009.pdb","seca_010.pdb"]

# Directory path for HADDOCK results --> CHANGE PATH 
with os.scandir('D:/RESULT_HADDOCK/') as ficheros:
    for fichero in ficheros:
        path_fichero = 'D:/RESULT_HADDOCK/' + fichero.name

        best_temp_row = []

        temp_matrix = []
        with os.scandir(path_fichero) as subficheros:

            for subfichero in subficheros:
                path_subfichero = path_fichero + "/"+ subfichero.name

                # List to store iRMSD values for comparison
                list_irmsd = []

                # Compute iRMSD using pdb2sql for each reference file
                for i in ref:
                    sim = StructureSimilarity(path_subfichero,"secA_models_split/"+ i)
                    irmsd = sim.compute_irmsd_pdb2sql()
                    list_irmsd.append(irmsd)
                
                # Append row with iRMSD values and calculated metrics to temporary matrix
                row = initial_matrix(fichero.name, path_subfichero, subfichero.name) + [min(list_irmsd)]
                #print(row)
                temp_matrix.append(row)

            # Determine the best row from temporary matrix and add to the best matrix
            best_temp_row = [fichero.name]+select_best_temp_row(temp_matrix)
            best_matrix.append(best_temp_row)
            print(best_temp_row)


# Uncomment the following line to save the matrix to a CSV file
#np.savetxt("OUT_NAME.csv",best_matrix ,delimiter =", ",fmt ='% s')

['C123_P0AEU0', 'cluster12', 308.5, -46.203, -15.686, 13.695, -26.114, 1358.722, -74.084, 0.446, 7.782, 5.912, 14.698, 2.127, 78.504, 7.11, 0.094]
['C124_P0AFH8', 'cluster14', 308.5, -45.916, -222.434, 73.366, -5.953, 1963.1, -89.019, 2.386, 18.808, 23.462, 22.671, 2.945, 351.774, 22.005, 0.377]
['C125_P0C8Z8', 'cluster6', 308.5, -45.481, -37.153, 16.002, -10.972, 1492.888, -62.283, 0.267, 5.397, 12.064, 16.831, 5.936, 144.124, 9.915, 0.052]
['C126_P24093', 'cluster3', 308.5, -48.319, -80.6, 22.738, -22.211, 1557.372, -84.376, 0.443, 5.176, 28.199, 20.035, 3.569, 163.37, 6.847, 0.042]
['C127_P28307', 'cluster1', 308.5, -46.479, -80.451, 33.215, -16.122, 1674.733, -75.37, 0.445, 4.325, 60.156, 13.518, 4.763, 92.521, 2.636, 0.219]
['C128_P28585', 'cluster3', 308.5, -48.591, -191.528, 22.898, -19.236, 2005.615, -103.843, 1.871, 9.948, 68.686, 22.471, 4.801, 238.224, 17.569, 0.223]
['C129_P30130', 'cluster11', 308.5, -37.944, -33.733, 42.203, -17.917, 1423.108, -58.387, 1.518, 4.514, 25.18